<a href="https://colab.research.google.com/github/1900690/Agro-Meteorological-Grid-Square-Data/blob/main/%E5%8F%82%E8%80%83/7_%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E9%9B%86%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

農研機構メッシュ農業気象データシステム(AMGSDS) 　ドキュメント<br>
チュートリアル　７ データの集計

# データの集計
　メッシュ農業気象データから月別値や年別値を得る方法を学びます。

　まず、以下のセルを実行し、このチュートリアルで使用するライブラリを読み込んでください。<br>
（実行後は、左端の青太線をクリックして畳んでおいてください）
* 図で日本語を使用したい場合は、**AMD_Tools4.pyの89-90行目**のコメントアウトを外して有効にしてください。その際、コメントをよく読みOSの違いにご注意ください。

In [ ]:
# このチュートリアルで使用するライブラリ
# ライブラリ（メッシュデータ処理のために必要）
#from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import xarray as xr
import AMD_Tools4 as amd

## 1 平均値、合計値の分布図
　関数GetMetDataはメッシュ農業気象データを3次元の配列で返します。漠然と「配列」と書きましたが、正確にはライブラリ Numpy が提供する ndarray というオブジェクトです。一般に、オブジェクトには、多くの場合、メソッドとよばれる計算機能が付随していて、そのオブジェクトならではの処理ができるようになっています。
　ndarray オブジェクトは数字の配列を一まとめに扱うオブジェクトですから、「ならでは」の関数として、合計や平均を計算するメソッド、sum や mean が用意されており、メッシュデータの合計や平均が簡単に計算できます。<br>
 https://numpy.org/doc/stable/reference/generated/numpy.sum.html <br>
 https://numpy.org/doc/stable/reference/generated/numpy.mean.html

　平均値や合計値の分布を求める例を、以下に二つ示します。

#### 例１
2023年9月の全天日射量分布図を作成する。

まず、この期間の日別全天日射量データを取得します。

In [ ]:
# 気象データの取得
nani = 'GSR'                          # 全天日射量
itsu = [ "2023-09-01", "2023-09-30" ] # 期間
doko = [ 34.666, 35.333, 136.000, 137.000]   #領域　愛知県周辺
data, tim, lat, lon = amd.GetMetData(nani, itsu, doko )

　気象データは、時間方向に30座標、緯度方向に80座標、経度方向に80座標の配列です。求めるのは、これら全部を足し合わせた答えではなく、それぞれのメッシュにおける時間についての合計なので、下のとおり、メソッドに引数 **axis=0** を与えます。メッシュ農業気象データは、時間(日付)、緯度、経度の順序で配列化されているので、集計を時間方向に限定する場合、最初（ゼロ番目）の次元という意味で0を指定します。

In [ ]:
# 日付方向の合計
SdSept = data.sum(axis=0)

In [ ]:
# 分布図の描画
amd.mapfig(SdSept,lat,lon,
       figtitle='Global solar radiation in Sep. 2023 [MJ/m2]')

#### 例２
　2023年の1月から12月までの月平均気温分布図を作成する。

　このような図は、_例１_ を for ループの中に入れて12回まわせば得ることができますが、１年分の気象データを全部取得し、その中から特定の月に属するデータだけを取り出して平均を取る方法で分布図を作成してみましょう。インデックスや時刻の取り扱をよく理解してください。

　まず気象データを取得します。

In [ ]:
# 気象データの取得
nani = 'TMP_mea'                      # 日平均気温
itsu = [ "2023-01-01", "2023-12-31" ] # 1年分
doko = [ 34.666, 35.333, 136.000, 137.000]    #愛知県周辺です。
data, tim, lat, lon = amd.GetMetData(nani, itsu, doko )

　次に、時刻の座標 **tim** を加工して、月の値の配列、すなわち、取得したデータの日付座標とおなじ数(365個)の要素を持ち、最初の31個は1、次の28個は2、・・、最後の31個は12が入っている整数の配列、を作ります。それは以下で作れます。

In [ ]:
# 月が格納されている配列
mm = np.array([dd.month for dd in tim])

In [ ]:
mm  # 確認したい場合実行

　配列 **tim** を構成する要素は Python標準の日時オブジェクトなので、属性 **month** により、月の数字(1月なら「1」)を取り出すことができます。上のスクリプトでは、この方法で tim のすべての要素から月の数字を取り出してリストにし(リスト内包標記の部分)、さらにそれを、ndarray オブジェクトに変換しています。<br>
 月の数字が格納された配列ができれば、これに関数 numpy.where を適用して、特定の月のデータが格納されているインデックスを得ることができます。

In [ ]:
# 指定した月のデータが格納されている要素のインデックスを得るスクリプト
month_i = 2  # 例えば2月
idx = np.where(mm == month_i)

In [ ]:
idx  # 確認したい場合実行

インデックスが分かれば、1年分の気温データから特定月のデータだけを抜き出すことができるので、これに平均を計算するメソッド **mean** を適用します。

In [ ]:
datai = data[idx]
Tmonmea = datai.mean(axis=0)

　これで2月の月平均気温のデータができました。可視化します。

In [ ]:
amd.mapfig(Tmonmea, lat, lon,
        figtitle=f'Mean air temperature in Month-{month_i}, 2023')

　細切れに説明したスクリプトをまとめ、さらにforループに納めたものを下に示します(データの取得部分は含めていません)。<br>
　これを実行すると12枚の分布図が表示されます。確認したら左端の青太線をクリックして図を畳んでおいてください。

In [ ]:
mm = np.array([dd.month for dd in tim])
months = sorted(list(set(mm)))  # mmから重複を除いてソートし[1,2,..,12]を作ってみた
for month_i in months: #[0:2]:
    idx = np.where(mm == month_i)
    Tmonmea = data[idx].mean(axis=0)
    amd.mapfig(Tmonmea, lat, lon,
            minmax = [-5,30],
            figtitle=f'Mean air temperature in Month-{month_i}, 2023')

## 2 pandas を利用した集計
　取得した気象データを、ライブラリ pandas が提供するオブジェクト DataFrame
 に変換するとより柔軟にデータを集計することができます。

#### 例
　茨城県つくば市における、2023年の各月の日最高気温の最大値を求める。

　まず、気象データを取得します。

In [ ]:
# 気象データの取得
nani = 'TMP_max'                      # 日平均気温
itsu = [ "2023-01-01", "2023-12-31" ] # 1年分
doko = [ 36.057, 36.057, 140.025, 140.025]    #アメダス つくば (茨城県つくば市内)
data, tim, lat, lon = amd.GetMetData(nani, itsu, doko )
data = data[:,0,0]

これを DataFrame オブジェクトにします。

In [ ]:
# DataFrame オブジェクトの作成
df = pd.DataFrame(data,
                  columns=['つくば'],
                  index=tim)

In [ ]:
df  # 確認したい場合実行

　これを集計するには、まず、メソッド **resample** でデータ全体を集計単位にグループ分けし、さらに、集計するメソッド を繋げてグループ内を集計します。<br>
　今回は月集計なので、reample には引数として 'MS' を与えます。月集計だけでなく週集計('W')や、_n_ 日間隔(_n_ D)などでの集計も可能です。詳しくは以下を参照してください。<br>
　(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)

　グループ分けしたそれぞれに対し、今回は最大値を取るので、集計のメソッドには **max** を使用します。

In [ ]:
# 集計
dfmonmax = df.resample("MS").max()
dfmonmax